# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [1]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 2.0974923478766567
 1.837365803810516
 2.5988149723914344

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [2]:
Alu = lu(A)

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.838149  1.0       0.0
 0.980231  0.345077  1.0
U factor:
3×3 Matrix{Float64}:
 0.992063   0.968402  0.137027
 0.0       -0.37128   0.450635
 0.0        0.0       0.515404

In [3]:
typeof(Alu)

LU{Float64, Matrix{Float64}, Vector{Int64}}

The different parts of the factorization can be extracted by accessing their special properties

In [4]:
Alu.P

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [5]:
Alu.L

3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.838149  1.0       0.0
 0.980231  0.345077  1.0

In [6]:
Alu.U

3×3 Matrix{Float64}:
 0.992063   0.968402  0.137027
 0.0       -0.37128   0.450635
 0.0        0.0       0.515404

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [7]:
A\b

3-element Vector{Float64}:
 0.9999999999999991
 1.0000000000000007
 1.000000000000001

In [8]:
Alu\b

3-element Vector{Float64}:
 0.9999999999999991
 1.0000000000000007
 1.000000000000001

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [9]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [10]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.612754   0.570492  -0.546874
 -0.513579  -0.81342   -0.273101
 -0.600641   0.11352    0.791419
R factor:
3×3 Matrix{Float64}:
 -1.61902  -1.31277   -0.858037
  0.0       0.287463  -0.290395
  0.0       0.0        0.407901

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [11]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.612754   0.570492  -0.546874
 -0.513579  -0.81342   -0.273101
 -0.600641   0.11352    0.791419

In [12]:
Aqr.R

3×3 Matrix{Float64}:
 -1.61902  -1.31277   -0.858037
  0.0       0.287463  -0.290395
  0.0       0.0        0.407901

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [13]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -0.5931046348413895
  0.677309883163626
  4.391143787138222
vectors:
3×3 Matrix{Float64}:
  0.457449  -0.606036   -0.650739
 -0.83814   -0.0493598  -0.543218
  0.297089   0.793905   -0.530522

The values and the vectors can be extracted from the Eigen type by special indexing

In [14]:
AsymEig.values

3-element Vector{Float64}:
 -0.5931046348413895
  0.677309883163626
  4.391143787138222

In [15]:
AsymEig.vectors

3×3 Matrix{Float64}:
  0.457449  -0.606036   -0.650739
 -0.83814   -0.0493598  -0.543218
  0.297089   0.793905   -0.530522

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [16]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
 1.0          -1.77636e-15   7.10543e-15
 4.44089e-16   1.0          -3.10862e-15
 2.44249e-15   3.10862e-15   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [20]:
n = 1000
A = randn(n,n);
A

1000×1000 Matrix{Float64}:
 -0.741343   -1.14458   -1.69625    …   0.837995   -0.373895   -0.508979
  1.6767      0.166299  -2.03025        1.58748     0.177972    1.20339
  1.35004     2.15188   -0.171319       0.713402    0.13388     1.15022
 -1.07959     0.511556   1.13811       -1.06416     0.190461    0.650069
 -0.130908    0.599576   1.14745        0.736755   -1.08062    -1.06662
  1.24824    -0.702492  -0.767667   …   1.14363     0.17461     1.84921
  0.511158   -1.02914   -0.246664      -0.733599    0.165667   -0.248381
  0.911032    0.475314   1.09863       -0.455832   -0.337482   -0.605183
  1.49158     0.863263  -0.936619       0.198465    0.0302847   0.344446
  1.1415     -0.068092  -0.861149      -1.16515     1.28732    -0.176018
 -1.40361    -1.19091   -0.487428   …   0.195803    0.877252    1.22616
  0.666796   -0.902936   1.57482       -0.534663   -0.984423   -0.173606
 -1.55906    -0.646322  -0.350794       0.779851   -0.290084    1.50264
  ⋮                           

Julia can often infer special matrix structure

In [21]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [22]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

0.5321198286693531

In [23]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [24]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [25]:
@time eigvals(Asym);

  0.582348 seconds (268.81 k allocations: 25.905 MiB, 1.81% gc time, 75.09% compilation time)


In [26]:
@time eigvals(Asym_noisy);

  0.774850 seconds (14 allocations: 7.928 MiB)


In [27]:
@time eigvals(Asym_explicit);

  0.260988 seconds (69.82 k allocations: 12.732 MiB, 40.71% compilation time)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [28]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.762275 seconds (491.92 k allocations: 214.438 MiB, 3.47% gc time, 36.90% compilation time)


6.484456970704247

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [29]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [30]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
 1//2  1//2   1//2
 2//5  7//10  3//10
 1//5  1//10  3//5

In [31]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
 3//2
 7//5
 9//10

In [32]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1//1
 1//1
 1//1

In [33]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}, Vector{Int64}}
L factor:
3×3 Matrix{Rational{BigInt}}:
 1//1   0//1  0//1
 4//5   1//1  0//1
 2//5  -1//3  1//1
U factor:
3×3 Matrix{Rational{BigInt}}:
 1//2  1//2    1//2
 0//1  3//10  -1//10
 0//1  0//1   11//30

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [36]:
using LinearAlgebra
A = [140 97 74 168 131; 97 106 89 131 36; 74 89 152 144 71; 168 131 144 54 142; 131 36 71 142 36]

5×5 Matrix{Int64}:
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36

In [40]:
A_eigv = eigvals(A)

5-element Vector{Float64}:
 -128.49322764802145
  -55.887784553056875
   42.7521672793189
   87.16111477514521
  542.4677301466143

In [41]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [44]:
A_diag = diagm(A_eigv)

5×5 Matrix{Float64}:
 -128.493    0.0      0.0      0.0       0.0
    0.0    -55.8878   0.0      0.0       0.0
    0.0      0.0     42.7522   0.0       0.0
    0.0      0.0      0.0     87.1611    0.0
    0.0      0.0      0.0      0.0     542.468

In [45]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]

LoadError: AssertionError: A_diag == [-128.493 0.0 0.0 0.0 0.0; 0.0 -55.8878 0.0 0.0 0.0; 0.0 0.0 42.7522 0.0 0.0; 0.0 0.0 0.0 87.1611 0.0; 0.0 0.0 0.0 0.0 542.468]

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [46]:
A_lowertri = LowerTriangular(A)

5×5 LowerTriangular{Int64, Matrix{Int64}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

In [47]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]